In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import librosa
from IPython.display import HTML

def extract_bpm(audio_path, default_bpm=120, baseline_bpm=90.0):
    """
    Load an audio file, extract the BPM, and compute a speed multiplier.
    """
    try:
        y, sr = librosa.load(audio_path)
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
        bpm = tempo if not isinstance(tempo, (np.ndarray, list)) else tempo[0]
    except Exception:
        bpm = default_bpm
    speed_multiplier = bpm / baseline_bpm
    return bpm, speed_multiplier

def animate_dance_sequence(data, t=None, skeleton=None, interval=33, speed_multiplier=1.0):
    """
    Animate a 3D dance sequence with BPM-based speed control, without extra visual frills.
    """
    if t is not None:
        data = data[:, :t, :]
    sequence = np.transpose(data, (1, 0, 2))
    n_frames, n_joints, _ = sequence.shape

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    scatter = ax.scatter(sequence[0, :, 0],
                         sequence[0, :, 1],
                         sequence[0, :, 2],
                         color='red', s=50)
    
    lines = []
    if skeleton:
        for i, j in skeleton:
            line, = ax.plot([sequence[0, i, 0], sequence[0, j, 0]],
                            [sequence[0, i, 1], sequence[0, j, 1]],
                            [sequence[0, i, 2], sequence[0, j, 2]],
                            color='blue', linewidth=2)
            lines.append(line)
    
    def update(frame):
        idx = int(frame * speed_multiplier) % n_frames
        scatter._offsets3d = (sequence[idx, :, 0],
                              sequence[idx, :, 1],
                              sequence[idx, :, 2])
        if skeleton:
            for line, (i, j) in zip(lines, skeleton):
                line.set_data([sequence[idx, i, 0], sequence[idx, j, 0]],
                              [sequence[idx, i, 1], sequence[idx, j, 1]])
                line.set_3d_properties([sequence[idx, i, 2], sequence[idx, j, 2]])
        return scatter, *lines

    anim = FuncAnimation(fig, update, frames=n_frames, interval=interval)
    plt.close(fig)
    return anim

    
